# Md Abrar Jahin

## Data Cleaning

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install emoji
import pandas as pd
import emoji
import re
import nltk
from nltk.corpus import stopwords
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
df_one = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Step_one.csv')
df_two = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Step_two.csv')
df_three = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Step_three.csv')

print(df_one.info()) #29934
# print(df_two.info()) #24702
# print(df_three.info()) #22742

In [ ]:
#merge all the datasets
all_df=[df_one, df_two, df_three]
all_df=pd.concat(all_df)
all_df = all_df.reset_index(drop=True)
print(len(all_df))
empty_tweet=all_df['tweet'].isna().value_counts()
print(empty_tweet) #no empty tweets

In [ ]:
all_df['clean_tweet']= all_df['tweet'].copy()
all_df=all_df.drop('id', axis=1)
all_df

In [ ]:
#handle emoji
def convert_emoji(text):
    text=[emoji.demojize(tw) for tw in text]  #run slowly
    new_df= pd.DataFrame(text, columns=['tweet'])
    return new_df
new_df=convert_emoji(all_df['clean_tweet'])
new_df

In [ ]:
all_df[['clean_tweet']]=new_df[['tweet']].copy()  
all_df

In [ ]:
#simple data cleaning
#reference:https://github.com/ugis22/analysing_twitter/blob/master/Jupyter%20Notebook%20files/Analysis%20of%20Twitter.ipynb
def preprocessing_one(df):
    #lower all characters
    df['clean_tweet'] = df['clean_tweet'].str.lower()
    #remove all the mentions: @username
    df['clean_tweet'] = df['clean_tweet'].replace(r'@\w+', '', regex=True)
    #remove all the links in the original tweets (start with "www" and "http" and "https")
    df['clean_tweet'] = df['clean_tweet'].replace(r'http\S+|rhttps\S+|rwww\S+', '', regex=True)   
    return df
#Notice: Remove punctuation and special characters after handling contraction words
preprocessing_one(all_df)

In [ ]:
nltk.download('wordnet')

In [ ]:
#Handling repeated characters
#reference: https://github.com/ugis22/analysing_twitter/blob/master/Jupyter%20Notebook%20files/Analysis%20of%20Twitter.ipynb
#https://stackoverflow.com/questions/3788870/how-to-check-if-a-word-is-an-english-word-with-python
from nltk.corpus import wordnet
#re.sub(pattern, repl, string, count): pattern(Eligible pattern)，repl(replace to...), string
def repeated_char(word):
    repeat_pattern = re.compile(r'(\w*)(\w)\2(\w*)')
    repl_word = r'\1\2\3'
    if wordnet.synsets(word):
        return word  #test for words existence
    #repl_new= repeat_pattern.sub(repl_word, word)
    repl_new = re.sub(repeat_pattern, repl_word, word)
    if repl_new != word:
        return repeated_char(repl_new)
    else:
        return repl_new
word1='loooove'
print(repeated_char(word1))

In [ ]:
def check_repeated (tweet):
    repeat_pattern = r'(\w*)(\w+)(\2)(\w*)'
    word_set = [''.join(i) for i in re.findall(repeat_pattern, tweet)]  #find all the words with repeated characters
    for word in word_set:
        if not wordnet.synsets(word):
            tweet=re.sub(word, repeated_char(word), tweet)
    return tweet
#test the function
tweet1='I looove you, soooo much'
print(check_repeated (tweet1))

In [ ]:
#replace words contraction
#reference:https://github.com/kiran-bal/Disaster_tweets_classifier/blob/2e6d648f5ef9cbe67024ad5cf032582fc4dc3a75/version2/notebooks/Disaster_tweet_classifier.ipynb
#re.sub(pattern, repl, string, count=0, flags=0): count=0: all matched will be replaced
contraction_dict=[(r'I\'m', 'I am'),(r'i\'m', 'i am'),(r'ain\'t', 'am not'),(r'(\w+)\'s', '\g<1> is'),(r'(\w+)\'re', '\g<1> are'),(r'(\w+)n\'t', '\g<1> not'),
                  (r'can\'t', 'cannot'),(r'won\'t', 'will not'), (r'wont', 'will not'), (r'(\w+)\'ll', '\g<1> will'), (r'(\w+)\'d', '\g<1> would'), (r'(\w+)\'ve', '\g<1> have'),
                 (r'I\’m', 'I am'),(r'i\’m', 'i am'),(r'ain\’t', 'am not'),(r'(\w+)\’s', '\g<1> is'),(r'(\w+)\’re', '\g<1> are'),(r'(\w+)n\’t', '\g<1> not'),
                  (r'can\’t', 'cannot'),(r'won\’t', 'will not'), (r'(\w+)\’ll', '\g<1> will'), (r'(\w+)\’d', '\g<1> would'), (r'(\w+)\’ve', '\g<1> have')]
#Notice: The quotation of some texts are not in English
def handle_contraction(text):
    patterns_set=[(re.compile(pattern), repl) for (pattern, repl) in contraction_dict]
    for (pattern, repl) in patterns_set:
        text=re.sub(pattern, repl, text)
    return text
#test
print(all_df['clean_tweet'][4])
print(handle_contraction(all_df['clean_tweet'][4]))

In [ ]:
def preprocessing_two(df):
    df['clean_tweet'] = df['clean_tweet'].apply(lambda x: check_repeated(x)) #remove repeated charaters
    df['clean_tweet'] = df['clean_tweet'].apply(lambda x: handle_contraction(x)) #handle constraction
    return df
all_df=preprocessing_two(all_df)
all_df

In [ ]:
#only English character
def replace_non_alphabetical(df):
    df['clean_tweet']=df['clean_tweet'].replace('[^a-zA-Z]',' ', regex=True)
    df['clean_tweet'] = df['clean_tweet'].replace('\s+', ' ', regex=True)
    return df
all_df=replace_non_alphabetical(all_df)
all_df

In [ ]:
#remove less than two-character words, but keep "no" if len(w)>2 or w=="no"
def short_words(df):
    df['clean_tweet'] = df['clean_tweet'].apply(lambda x: ' '.join([w for w in x.split() if len(w)>2 or w=="no"]))
    return df
all_df=short_words(all_df)
all_df.head()

In [ ]:
#remove stopwords
#can't remove words like "not" or "no"

In [ ]:
#create own stopwords
my_stopwords = ['a', "a's", 'able', 'about', 'above', 'according', 'accordingly', 'across', 'actually', 'after', 'afterwards', 'again', 'against', "ain't", 'all', 'allow', 'allows', 'almost', 'alone', 'along', 'already', 'also', 'although', 'always', 'am', 'among', 'amongst', 'an', 'and', 'another', 'any', 'anybody', 'anyhow', 'anyone', 'anything', 'anyway', 'anyways', 'anywhere', 'apart', 'appear', 'appreciate', 'appropriate', 'are', "aren't", 'around', 'as', 'aside', 'ask', 'asking', 'associated', 'at', 'available', 'away', 'awfully', 'b', 'be', 'became', 'because', 'become', 'becomes', 'becoming', 'been', 'before', 'beforehand', 'behind', 'being', 'believe', 'below', 'beside', 'besides', 'best', 'better', 'between', 'beyond', 'both', 'brief', 'but', 'by', 'c', "c'mon", "c's", 'came', 'can', "can't", 'cannot', 'cant', 'cause', 'causes', 'certain', 'certainly', 'changes', 'clearly', 'co', 'com', 'come', 'comes', 'concerning', 'consequently', 'consider', 'considering', 'contain', 'containing', 'contains', 'corresponding', 'could', "couldn't", 'course', 'currently', 'd', 'definitely', 'described', 'despite', 'did', "didn't", 'different', 'do', 'does', "doesn't", 'doing', "don't", 'done', 'down', 'downwards', 'during', 'e', 'each', 'edu', 'eg', 'eight', 'either', 'else', 'elsewhere', 'enough', 'entirely', 'especially', 'et', 'etc', 'even', 'ever', 'every', 'everybody', 'everyone', 'everything', 'everywhere', 'ex', 'exactly', 'example', 'except', 'f', 'far', 'few', 'fifth', 'first', 'five', 'followed', 'following', 'follows', 'for', 'former', 'formerly', 'forth', 'four', 'from', 'further', 'furthermore', 'g', 'get', 'gets', 'getting', 'given', 'gives', 'go', 'goes', 'going', 'gone', 'got', 'gotten', 'greetings', 'h', 'had', "hadn't", 'happens', 'hardly', 'has', "hasn't", 'have', "haven't", 'having', 'he', "he's", 'hello', 'help', 'hence', 'her', 'here', "here's", 'hereafter', 'hereby', 'herein', 'hereupon', 'hers', 'herself', 'hi', 'him', 'himself', 'his', 'hither', 'hopefully', 'how', 'howbeit', 'however', 'i', "i'd", "i'll", "i'm", "i've", 'ie', 'if', 'ignored', 'immediate', 'in', 'inasmuch', 'inc', 'indeed', 'indicate', 'indicated', 'indicates', 'inner', 'insofar', 'instead', 'into', 'inward', 'is', "isn't", 'it', "it'd", "it'll", "it's", 'its', 'itself', 'j', 'just', 'k', 'keep', 'keeps', 'kept', 'know', 'knows', 'known', 'l', 'last', 'lately', 'later', 'latter', 'latterly', 'least', 'less', 'lest', 'let', "let's", 'like', 'liked', 'likely', 'little', 'look', 'looking', 'looks', 'ltd', 'm', 'mainly', 'many', 'may', 'maybe', 'me', 'mean', 'meanwhile', 'merely', 'might', 'more', 'moreover', 'most', 'mostly', 'much', 'must', 'my', 'myself', 'n', 'name', 'namely', 'nd', 'near', 'nearly', 'necessary', 'need', 'needs', 'neither', 'never', 'nevertheless', 'new', 'next', 'nine', 'normally', 'nothing', 'novel', 'now', 'nowhere', 'o', 'obviously', 'of', 'off', 'often', 'oh', 'ok', 'okay', 'old', 'on', 'once', 'one', 'ones', 'only', 'onto', 'or', 'other', 'others', 'otherwise', 'ought', 'our', 'ours', 'ourselves', 'out', 'outside', 'over', 'overall', 'own', 'p', 'particular', 'particularly', 'per', 'perhaps', 'placed', 'please', 'plus', 'possible', 'presumably', 'probably', 'provides', 'q', 'que', 'quite', 'qv', 'r', 'rather', 'rd', 're', 'really', 'reasonably', 'regarding', 'regardless', 'regards', 'relatively', 'respectively', 'right', 's', 'said', 'same', 'saw', 'say', 'saying', 'says', 'second', 'secondly', 'see', 'seeing', 'seem', 'seemed', 'seeming', 'seems', 'seen', 'self', 'selves', 'sensible', 'sent', 'serious', 'seriously', 'seven', 'several', 'shall', 'she', 'should', "shouldn't", 'since', 'six', 'so', 'some', 'somebody', 'somehow', 'someone', 'something', 'sometime', 'sometimes', 'somewhat', 'somewhere', 'soon', 'sorry', 'specified', 'specify', 'specifying', 'still', 'sub', 'such', 'sup', 'sure', 't', "t's", 'take', 'taken', 'tell', 'tends', 'th', 'than', 'thank', 'thanks', 'thanx', 'that', "that's", 'thats', 'the', 'their', 'theirs', 'them', 'themselves', 'then', 'thence', 'there', "there's", 'thereafter', 'thereby', 'therefore', 'therein', 'theres', 'thereupon', 'these', 'they', "they'd", "they'll", "they're", "they've", 'think', 'third', 'this', 'thorough', 'thoroughly', 'those', 'though', 'three', 'through', 'throughout', 'thru', 'thus', 'to', 'together', 'too', 'took', 'toward', 'towards', 'tried', 'tries', 'truly', 'try', 'trying', 'twice', 'two', 'u', 'un', 'under', 'unfortunately', 'unless', 'unlikely', 'until', 'unto', 'up', 'upon', 'us', 'use', 'used', 'useful', 'uses', 'using', 'usually', 'uucp', 'v', 'value', 'various', 'very', 'via', 'viz', 'vs', 'w', 'want', 'wants', 'was', "wasn't", 'way', 'we', "we'd", "we'll", "we're", "we've", 'welcome', 'well', 'went', 'were', "weren't", 'what', "what's", 'whatever', 'when', 'whence', 'whenever', 'where', "where's", 'whereafter', 'whereas', 'whereby', 'wherein', 'whereupon', 'wherever', 'whether', 'which', 'while', 'whither', 'who', "who's", 'whoever', 'whole', 'whom', 'whose', 'why', 'will', 'willing', 'wish', 'with', 'within', 'without', "won't", 'wonder', 'would', 'would', "wouldn't", 'x', 'y', 'yes', 'yet', 'you', "you'd", "you'll", "you're", "you've", 'your', 'yours', 'yourself', 'yourselves', '']
print(my_stopwords)

def remove_stopwords(df):
    df['clean_tweet']=df['clean_tweet'].apply(lambda x: ' '.join([word for word in x.split() if word not in my_stopwords]))
    return df
all_df=remove_stopwords(all_df)
all_df
# all_df.to_csv('clean_datasets/all_clean_1_test.csv',index = False, encoding='utf_8_sig')

In [ ]:
#remove empty tweets
def remove_empty(df):
    df=df[df['clean_tweet']!='']
    return df
all_df=remove_empty(all_df)
all_df=all_df.reset_index(drop=True)
all_df

In [ ]:
#split the clean datasets into three steps
all_df.to_csv('all_clean_1.csv',index = False, encoding='utf_8_sig')

In [ ]:
#clean_df = pd.read_csv('/kaggle/input/uk-twitter-covid19-dataset/all_clean_1.csv')
clean_df=all_df
clean_df['created_at'] = clean_df['created_at'].str.replace(r'[^\x00-\x7F]+', '')

In [ ]:
clean_df

In [ ]:
clean_df['created_at'] = pd.to_datetime(clean_df['created_at'] , utc=True).dt.date
#clean_df.to_csv('clean_datasets/all_clean_1.csv',index = False, encoding='utf_8_sig')
clean_df

In [ ]:
import datetime
clean_step_one = clean_df[(clean_df['created_at']) < datetime.date(2021,3,8)].reset_index(drop=True)
print("step_one:",len(clean_step_one))

In [ ]:
clean_step_two = clean_df[((clean_df['created_at']) >= datetime.date(2021,3,8)) & ((clean_df['created_at']) < datetime.date(2021,5,17))].reset_index(drop=True)
print("step_two:",len(clean_step_two))
# clean_step_two

In [ ]:
clean_step_three= clean_df[((clean_df['created_at']) >= datetime.date(2021,5,17)) & ((clean_df['created_at']) <= datetime.date(2021,7,18))].reset_index(drop=True)
print("step_three:",len(clean_step_three))
# clean_step_three

In [ ]:
#save as csv
clean_step_one.to_csv('clean_step1.csv',index = False, encoding='utf_8_sig')
clean_step_two.to_csv('clean_step2.csv',index = False, encoding='utf_8_sig')
clean_step_three.to_csv('clean_step3.csv',index = False, encoding='utf_8_sig')

In [ ]:
len(clean_step_one)+len(clean_step_two)+len(clean_step_three)

In [ ]:
clean_all=[clean_step_one,clean_step_two,clean_step_three]
clean_all=pd.concat(clean_all)
clean_all_dfs=clean_all.reset_index(drop=True)
# new_dfs.to_csv("unsupervised_datasets/lexicon_all.csv")
clean_all.to_csv('clean_all.csv',index = False, encoding='utf_8_sig') #final clean datasets
clean_all

In [ ]:
#The number of tweets collected in each city
#London
London_df = clean_all[clean_all['near'].str.contains('London|london')]
London_df = London_df.reset_index(drop=True)
print("Total tweets in London:",len(London_df))
#Birmingham
Birmingham_df = clean_all[clean_all['near'].str.contains('Birmingham|birmingham')]
Birmingham_df = Birmingham_df.reset_index(drop=True)
print("Total tweets in Birmingham:",len(Birmingham_df))
#Bristol
Bristol_df = clean_all[clean_all['near'].str.contains('Bristol|Bristol')]
Bristol_df = Bristol_df.reset_index(drop=True)
print("Total tweets in Bristol:",len(Bristol_df))
#Leeds
Leeds_df = clean_all[clean_all['near'].str.contains('Leeds|leeds')]
Leeds_df = Leeds_df.reset_index(drop=True)
print("Total tweets in Leeds:",len(Leeds_df))
#Liverpool
Liverpool_df = clean_all[clean_all['near'].str.contains('Liverpool|liverpool')]
Liverpool_df = Liverpool_df.reset_index(drop=True)
print("Total tweets in Liverpool:",len(Liverpool_df))
#Manchester
Manchester_df = clean_all[clean_all['near'].str.contains('Manchester|manchester')]
Manchester_df = Manchester_df.reset_index(drop=True)
print("Total tweets in Manchester:",len(Manchester_df))
#Newcastle
Newcastle_df = clean_all[clean_all['near'].str.contains('Newcastle|newcastle')]
Newcastle_df = Newcastle_df.reset_index(drop=True)
print("Total tweets in Newcastle:",len(Newcastle_df))
#Nottingham
Nottingham_df = clean_all[clean_all['near'].str.contains('Nottingham|nottingham')]
Nottingham_df = Nottingham_df.reset_index(drop=True)
print("Total tweets in Nottingham:",len(Nottingham_df))
#Sheffield
Sheffield_df = clean_all[clean_all['near'].str.contains('Sheffield|sheffield')]
Sheffield_df = Sheffield_df.reset_index(drop=True)
print("Total tweets in Sheffield:",len(Sheffield_df))
#Southampton
Southampton_df = clean_all[clean_all['near'].str.contains('Southampton|southampton')]
Southampton_df = Southampton_df.reset_index(drop=True)
print("Total tweets in Southampton:",len(Southampton_df))
print(len(London_df)+len(Birmingham_df)+len(Bristol_df)+len(Leeds_df)+len(Liverpool_df)+len(Manchester_df)+len(Newcastle_df)+len(Nottingham_df)+len(Sheffield_df)+len(Southampton_df))

In [ ]:
cities = {"city": ["London", "Manchester", "Birmingham", "Liverpool", "Leeds", "Bristol", "Sheffield", "Nottingham","Southampton", "Newcastle"],
           "total number":[len(London_df),  len(Manchester_df), len(Birmingham_df), len(Liverpool_df), len(Leeds_df), len(Bristol_df), len(Sheffield_df),len(Nottingham_df),len(Southampton_df), len(Newcastle_df)]}
city_frame  = pd.DataFrame(data = cities)
city_frame.to_csv('all_cities.csv',index = False)
city_frame

In [ ]:
np.arange(len(city_frame['total number'])) 

In [ ]:
#reference: https://stackoverflow.com/questions/60212294/how-to-add-labels-to-a-horizontal-bar-chart-in-matplotlib
plt.rcParams["figure.figsize"] = (12,6)
fig, ax = plt.subplots()   

width = 0.75 # bar's width 
x_loc = np.arange(len(city_frame['total number']))  # the x locations
ax.barh(x_loc, city_frame['total number'], width, color="slateblue")
ax.set_yticks(x_loc+width/2)
ax.set_yticklabels(city_frame['city'], minor=False)

_, xmax = plt.xlim()
plt.xlim(0, xmax+300)
for i, v in enumerate(city_frame['total number']):
    ax.text(v + 100, i, str(v), color='black', fontsize=10, ha='left', va='center')
    
plt.title('The distribution of tweets number')
plt.xlabel('Count')
plt.ylabel('City')

# plt.show()
plt.savefig("tweets distribution")

In [ ]:
#The number of tweets collected in each city
#step 1
#London
London_df = clean_step_one[clean_step_one['near'].str.contains('London|london')]
London_df = London_df.reset_index(drop=True)
print("Step1 tweets in London:",len(London_df))
#Birmingham
Birmingham_df = clean_step_one[clean_step_one['near'].str.contains('Birmingham|birmingham')]
Birmingham_df = Birmingham_df.reset_index(drop=True)
print("Step1 tweets in Birmingham:",len(Birmingham_df))
#Bristol
Bristol_df = clean_step_one[clean_step_one['near'].str.contains('Bristol|Bristol')]
Bristol_df = Bristol_df.reset_index(drop=True)
print("Step1 tweets in Bristol:",len(Bristol_df))
#Leeds
Leeds_df = clean_step_one[clean_step_one['near'].str.contains('Leeds|leeds')]
Leeds_df = Leeds_df.reset_index(drop=True)
print("Step1 tweets in Leeds:",len(Leeds_df))
#Liverpool
Liverpool_df = clean_step_one[clean_step_one['near'].str.contains('Liverpool|liverpool')]
Liverpool_df = Liverpool_df.reset_index(drop=True)
print("Step1 tweets in Liverpool:",len(Liverpool_df))
#Manchester
Manchester_df = clean_step_one[clean_step_one['near'].str.contains('Manchester|manchester')]
Manchester_df = Manchester_df.reset_index(drop=True)
print("Step1 tweets in Manchester:",len(Manchester_df))
#Newcastle
Newcastle_df = clean_step_one[clean_step_one['near'].str.contains('Newcastle|newcastle')]
Newcastle_df = Newcastle_df.reset_index(drop=True)
print("Step1 tweets in Newcastle:",len(Newcastle_df))
#Nottingham
Nottingham_df = clean_step_one[clean_step_one['near'].str.contains('Nottingham|nottingham')]
Nottingham_df = Nottingham_df.reset_index(drop=True)
print("Step1 tweets in Nottingham:",len(Nottingham_df))
#Sheffield
Sheffield_df = clean_step_one[clean_step_one['near'].str.contains('Sheffield|sheffield')]
Sheffield_df = Sheffield_df.reset_index(drop=True)
print("Step1 tweets in Sheffield:",len(Sheffield_df))
#Southampton
Southampton_df = clean_step_one[clean_step_one['near'].str.contains('Southampton|southampton')]
Southampton_df = Southampton_df.reset_index(drop=True)
print("Step1 tweets in Southampton:",len(Southampton_df))

In [ ]:
#step 2
#The number of tweets collected in each city
#London
London_df = clean_step_two[clean_step_two['near'].str.contains('London|london')]
London_df = London_df.reset_index(drop=True)
print("Step2 tweets in London:",len(London_df))
#Birmingham
Birmingham_df = clean_step_two[clean_step_two['near'].str.contains('Birmingham|birmingham')]
Birmingham_df = Birmingham_df.reset_index(drop=True)
print("Step2 tweets in Birmingham:",len(Birmingham_df))
#Bristol
Bristol_df = clean_step_two[clean_step_two['near'].str.contains('Bristol|Bristol')]
Bristol_df = Bristol_df.reset_index(drop=True)
print("Step2 tweets in Bristol:",len(Bristol_df))
#Leeds
Leeds_df = clean_step_two[clean_step_two['near'].str.contains('Leeds|leeds')]
Leeds_df = Leeds_df.reset_index(drop=True)
print("Step2 tweets in Leeds:",len(Leeds_df))
#Liverpool
Liverpool_df = clean_step_two[clean_step_two['near'].str.contains('Liverpool|liverpool')]
Liverpool_df = Liverpool_df.reset_index(drop=True)
print("Step2 tweets in Liverpool:",len(Liverpool_df))
#Manchester
Manchester_df = clean_step_two[clean_step_two['near'].str.contains('Manchester|manchester')]
Manchester_df = Manchester_df.reset_index(drop=True)
print("Step2 tweets in Manchester:",len(Manchester_df))
#Newcastle
Newcastle_df = clean_step_two[clean_step_two['near'].str.contains('Newcastle|newcastle')]
Newcastle_df = Newcastle_df.reset_index(drop=True)
print("Step2 tweets in Newcastle:",len(Newcastle_df))
#Nottingham
Nottingham_df = clean_step_two[clean_step_two['near'].str.contains('Nottingham|nottingham')]
Nottingham_df = Nottingham_df.reset_index(drop=True)
print("Step2 tweets in Nottingham:",len(Nottingham_df))
#Sheffield
Sheffield_df = clean_step_two[clean_step_two['near'].str.contains('Sheffield|sheffield')]
Sheffield_df = Sheffield_df.reset_index(drop=True)
print("Step2 tweets in Sheffield:",len(Sheffield_df))
#Southampton
Southampton_df = clean_step_two[clean_step_two['near'].str.contains('Southampton|southampton')]
Southampton_df = Southampton_df.reset_index(drop=True)
print("Step2 tweets in Southampton:",len(Southampton_df))

In [ ]:
#step 3
#The number of tweets collected in each city
#London
London_df = clean_step_three[clean_step_three['near'].str.contains('London|london')]
London_df = London_df.reset_index(drop=True)
print("Step3 tweets in London:",len(London_df))
#Birmingham
Birmingham_df = clean_step_three[clean_step_three['near'].str.contains('Birmingham|birmingham')]
Birmingham_df = Birmingham_df.reset_index(drop=True)
print("Step3 tweets in Birmingham:",len(Birmingham_df))
#Bristol
Bristol_df = clean_step_three[clean_step_three['near'].str.contains('Bristol|Bristol')]
Bristol_df = Bristol_df.reset_index(drop=True)
print("Step3 tweets in Bristol:",len(Bristol_df))
#Leeds
Leeds_df = clean_step_three[clean_step_three['near'].str.contains('Leeds|leeds')]
Leeds_df = Leeds_df.reset_index(drop=True)
print("Step3 tweets in Leeds:",len(Leeds_df))
#Liverpool
Liverpool_df = clean_step_three[clean_step_three['near'].str.contains('Liverpool|liverpool')]
Liverpool_df = Liverpool_df.reset_index(drop=True)
print("Step3 tweets in Liverpool:",len(Liverpool_df))
#Manchester
Manchester_df = clean_step_three[clean_step_three['near'].str.contains('Manchester|manchester')]
Manchester_df = Manchester_df.reset_index(drop=True)
print("Step3 tweets in Manchester:",len(Manchester_df))
#Newcastle
Newcastle_df = clean_step_three[clean_step_three['near'].str.contains('Newcastle|newcastle')]
Newcastle_df = Newcastle_df.reset_index(drop=True)
print("Step3 tweets in Newcastle:",len(Newcastle_df))
#Nottingham
Nottingham_df = clean_step_three[clean_step_three['near'].str.contains('Nottingham|nottingham')]
Nottingham_df = Nottingham_df.reset_index(drop=True)
print("Step3 tweets in Nottingham:",len(Nottingham_df))
#Sheffield
Sheffield_df = clean_step_three[clean_step_three['near'].str.contains('Sheffield|sheffield')]
Sheffield_df = Sheffield_df.reset_index(drop=True)
print("Step3 tweets in Sheffield:",len(Sheffield_df))
#Southampton
Southampton_df = clean_step_three[clean_step_three['near'].str.contains('Southampton|southampton')]
Southampton_df = Southampton_df.reset_index(drop=True)
print("Step3 tweets in Southampton:",len(Southampton_df))

In [ ]:
#stack bar chart
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
    
Steps_df = pd.DataFrame([['Stage1', 12870, 3010, 1572, 1857, 2372, 3796, 239, 1315, 1604, 1287], 
                   ['Stage2', 12945, 1905, 1003, 1214, 1652, 2838, 249, 942, 1185, 755], 
                   ['Stage3', 11863, 1793, 1187, 1170, 1506, 2559, 364, 735, 931, 612]],
                  columns=['Stages', 'London','Birmingham','Bristol','Leeds','Liverpool','Manchester','Newcastle','Nottingham','Sheffield','Southampton'])
Steps_df.to_csv('three_steps_cities.csv',index = False, encoding='utf_8_sig')
Steps_df

In [ ]:
# plt.xticks(rotation=0, ha='center')
Steps_df.plot(x='Stages', kind='bar', stacked=True,
        title='Stacked Bar Graph by Different Steps')
plt.xticks(rotation=0, ha='center')
plt.savefig("stacked bar graph")

## Further Cleaning and lexicon-based method

In [ ]:
import pandas as pd
!pip install vaderSentiment
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from wordcloud import WordCloud
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
df=clean_all
#df=pd.read_csv('/kaggle/input/uk-twitter-covid19-dataset/clean_all.csv')
df.head()
len(df)

In [ ]:
#tokenization
def tweet_token(text):
    words_set=text.str.split()
    tokens=[word for word in words_set]
    return tokens

df['tokens']=tweet_token(df['clean_tweet'])
df.head()

In [ ]:
#lemmatize and stemming
import nltk
word_tokenizer = nltk.tokenize.WhitespaceTokenizer()
word_lemmatizer = nltk.stem.WordNetLemmatizer()

def lemmatize_text(text):
    return [word_lemmatizer.lemmatize(word) for word in word_tokenizer.tokenize(text)]
df['lemma']=df['clean_tweet'].apply(lemmatize_text)
df.head()

In [ ]:
#lemma_sentence
df['lemma_sentence'] = df['lemma'].apply(lambda x: ' '.join(x))
df.head()

In [ ]:
#POS for clean tweets
#reference:
#https://www.machinelearningplus.com/nlp/lemmatization-examples-python/
#https://stackoverflow.com/questions/51267166/lemmatization-pandas-python
nltk.download('punkt')
from nltk import pos_tag
nltk.download('averaged_perceptron_tagger')
from nltk.tokenize import word_tokenize
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer
stem_lemmatizer = WordNetLemmatizer()

def convert_wordnet_tag(tag):
    if tag.startswith('J'):
        return wordnet.ADJ
    elif tag.startswith('V'):
        return wordnet.VERB
    elif tag.startswith('N'):
        return wordnet.NOUN
    elif tag.startswith('R'):
        return wordnet.ADV
    else:
        return None

def pos_tag_set(tweet):
    tagged_words = nltk.pos_tag(nltk.word_tokenize(tweet))
    new_tag=[]
    for word, tag in tagged_words:
        new_tag.append(tuple([word, convert_wordnet_tag(tag)]))
    return new_tag
df['pos_tag']=df['clean_tweet'].apply(pos_tag_set)
df

In [ ]:
#create lemma sentence with pos-tags
def handle_lemma(pos_tweet):
    lemma_set = " "
    for word, pos in pos_tweet:
        if not pos: 
            lemma = word
            lemma_set = lemma_set + " " + lemma
        else:  
            lemma = stem_lemmatizer.lemmatize(word, pos=pos)
            lemma_set = lemma_set + " " + lemma
    return lemma_set
df['pos_tag'].apply(handle_lemma)
    
df['lemma_sentence(with POS)'] = df['pos_tag'].apply(handle_lemma)
df

In [ ]:
#save as csv
df.to_csv('further_clean_all.csv',index = False, encoding='utf_8_sig')

In [ ]:
further_clean_all = df

In [ ]:
#split into 3 steps
import datetime
df['created_at'] = pd.to_datetime(df['created_at'] , utc=True).dt.date
further_step_one = df[(df['created_at']) < datetime.date(2021,3,8)].reset_index(drop=True)
print("step_one:",len(further_step_one))

In [ ]:
further_step_two = df[((df['created_at']) >= datetime.date(2021,3,8)) & ((df['created_at']) < datetime.date(2021,5,17))].reset_index(drop=True)
print("step_two:",len(further_step_two))

In [ ]:
further_step_three= df[((df['created_at']) >= datetime.date(2021,5,17)) & ((df['created_at']) <= datetime.date(2021,7,18))].reset_index(drop=True)
print("step_three:",len(further_step_three))
len(further_step_one)+len(further_step_two)+len(further_step_three)

In [ ]:
further_step_one.to_csv('further_clean_step1.csv',index = False, encoding='utf_8_sig')
further_step_two.to_csv('further_clean_step2.csv',index = False, encoding='utf_8_sig')
further_step_three.to_csv('further_clean_step3.csv',index = False, encoding='utf_8_sig')

In [ ]:
#sentiwordnet analysis
#reference: https://github.com/harika-bonthu/Lexicon-based-SentimentAnalysis/blob/main/lexicon_based_sentiment_analysis.ipynb
nltk.download('sentiwordnet')
from nltk.corpus import sentiwordnet as swn
from nltk.stem import WordNetLemmatizer
wordnet_lemmatizer = WordNetLemmatizer()
#和之前的方法结果 有差距

def sentiwordnetanalysis(pos_data):
    sentiment = 0
    tokens_count = 0
    for word, pos in pos_data:
        if not pos:
            continue
        lemma = wordnet_lemmatizer.lemmatize(word, pos=pos)
        if not lemma:
            continue
        
        synsets = wordnet.synsets(lemma, pos=pos)
        if not synsets:
            continue

        synset = synsets[0] #only take the most common meaning
        swn_synset = swn.senti_synset(synset.name())
        sentiment += swn_synset.pos_score() - swn_synset.neg_score()
        tokens_count += 1
    if not tokens_count:
        return 0
    else:
        return sentiment

df['sentiword_analysis']=df['pos_tag'].apply(sentiwordnetanalysis)
df

In [ ]:
#VADER--low speed--use further clean datasets
sentiment_analyzer = SentimentIntensityAnalyzer()
def vaderSentiment_method(df):
    sentiment_analyzer = SentimentIntensityAnalyzer()
    snt_score = sentiment_analyzer.polarity_scores(df['lemma_sentence(with POS)'])
    return snt_score['compound'] 

df['vader_score'] = df.apply(vaderSentiment_method, axis=1)
df

In [ ]:
#Textblob
from textblob import TextBlob

def Polarity_score(tweet):
    return TextBlob(tweet).sentiment.polarity

def Subjectivity_score(tweet):
    return TextBlob(tweet).sentiment.subjectivity

df['textblob_polarity'] = df['lemma_sentence(with POS)'].apply(Polarity_score) 
df

In [ ]:
##save as csv
df.to_csv('unsupervised_lexicon_all.csv',index = False, encoding='utf_8_sig')

In [ ]:
#split into 3 steps
import datetime
df['created_at'] = pd.to_datetime(df['created_at'] , utc=True).dt.date
lexicon_step_one = df[(df['created_at']) < datetime.date(2021,3,8)].reset_index(drop=True)
print("step_one:",len(lexicon_step_one))

In [ ]:
lexicon_step_two = df[((df['created_at']) >= datetime.date(2021,3,8)) & ((df['created_at']) < datetime.date(2021,5,17))].reset_index(drop=True)
print("step_two:",len(lexicon_step_two))

In [ ]:
lexicon_step_three= df[((df['created_at']) >= datetime.date(2021,5,17)) & ((df['created_at']) <= datetime.date(2021,7,18))].reset_index(drop=True)
print("step_three:",len(lexicon_step_three))

In [ ]:
#merge three datasets
dfs=[lexicon_step_one,lexicon_step_two,lexicon_step_three]
dfs=pd.concat(dfs)
dfs=dfs.reset_index(drop=True)

In [ ]:
lexicon_step_one.to_csv('unsupervised_lexicon_step1.csv',index = False, encoding='utf_8_sig')
lexicon_step_two.to_csv('unsupervised_lexicon_step2.csv',index = False, encoding='utf_8_sig')
lexicon_step_three.to_csv('unsupervised_lexicon_step3.csv',index = False, encoding='utf_8_sig')

## Initial data analysis

In [ ]:
import pandas as pd
from wordcloud import WordCloud
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

In [ ]:
#df=pd.read_csv('/content/further_clean_all.csv')
df = further_clean_all
df.head()

In [ ]:
#word cloud map
#https://www.datacamp.com/community/tutorials/wordcloud-python
def word_cloud(words):
    wordcloud = WordCloud(width=800, height=600, random_state=21, relative_scaling=0.5, background_color="white").generate(words)
    plt.figure(figsize=(8, 6))
    plt.imshow(wordcloud, interpolation="bilinear")
    plt.axis('off')
    plt.show()
word_set = ' '.join([tweet_word for tweet_word in df['lemma_sentence(with POS)']])
word_cloud(word_set)
plt.savefig('word_cloud.jpg')
#vaccine is a hot topic

In [ ]:
df['lemma'].value_counts()[0:10]

In [ ]:
#Word frequency
from nltk.tokenize import word_tokenize
from collections import Counter
from collections import Counter
texts = df['lemma_sentence(with POS)']
word_counts = Counter(word_tokenize('\n'.join(texts)))
word_top=word_counts.most_common(n=20)
print(word_top)
# count_sum=

In [ ]:
#(5)
count_all = df['lemma_sentence(with POS)'].str.len().sum()
print(count_all)

words=[count[0] for count in word_top]
frac_value=[int(count[1])/count_all for count in  word_top]
words=words[: :-1]
frac_value=sorted(frac_value)

#plot line chart
plt.figure(figsize=(10, 6))
plt.plot(words,frac_value)
plt.xticks(rotation=40, fontsize=13)
plt.xlabel('Words',fontsize=15)
plt.ylabel('Word Fraction',fontsize=15)
plt.title('Word Frequency Chart',fontsize=16)

## Data analysis for lexicon-based methods

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
#df1=pd.read_csv('/kaggle/input/uk-twitter-covid19-dataset/unsupervised_lexicon_step1.csv')
#df2=pd.read_csv('/kaggle/input/uk-twitter-covid19-dataset/unsupervised_lexicon_step2.csv')
#df3=pd.read_csv('/kaggle/input/uk-twitter-covid19-dataset/unsupervised_lexicon_step3.csv')
#dfs=pd.read_csv('/kaggle/input/uk-twitter-covid19-dataset/unsupervised_lexicon_all.csv')
df1=lexicon_step_one
df2=lexicon_step_two
df3=lexicon_step_three
dfs=df
print(len(dfs))
print(len(df1)+len(df2)+len(df3))

In [ ]:
#sentiment for step1
#for textblob and sentiword
def senti_label(score):
    if score < 0:
        return -1
    elif score == 0:
        return 0
    else:
        return 1

In [ ]:
#for VADER
def senti_label_2(score):
    if score <= -0.05:
        return -1
    elif score >= 0.05:
        return 1
    else:
        return 0

In [ ]:
#Step1
#using senti_label
df1['senti_textblob'] = df1['textblob_polarity'].apply(senti_label)
df1['senti_wordnet'] = df1['sentiword_analysis'].apply(senti_label)
df1.to_csv('supervised_lexicon_step1.csv')
#counting
blob_1=df1.groupby('senti_textblob').count()
print("textblob_step1:",blob_1["textblob_polarity"])

wordnet_1=df1.groupby('senti_wordnet').count()
print("wordnet_step1:",wordnet_1["sentiword_analysis"])

#using senti_label_2
df1['senti_vader'] = df1['vader_score'].apply(senti_label_2)
df1
vader_1=df1.groupby('senti_vader').count()
print('vader_step1:',vader_1["vader_score"])
df1
df1.to_csv("supervised_lexicon_step1.csv")

In [ ]:
supervised_lexicon_step1 = df1

In [ ]:
#Step2
#using senti_label
df2['senti_textblob'] = df2['textblob_polarity'].apply(senti_label)
df2['senti_wordnet'] = df2['sentiword_analysis'].apply(senti_label)
df2
#counting
blob_2=df2.groupby('senti_textblob').count()
print("textblob_step2:",blob_2["textblob_polarity"])

wordnet_2=df2.groupby('senti_wordnet').count()
print("wordnet_step2:",wordnet_2["sentiword_analysis"])

#using senti_label_2
df2['senti_vader'] = df2['vader_score'].apply(senti_label_2)
df2
vader_2=df2.groupby('senti_vader').count()
print('vader_step2:',vader_2["vader_score"])

In [ ]:
#Step3
#using senti_label
df3['senti_textblob'] = df3['textblob_polarity'].apply(senti_label)
df3['senti_wordnet'] = df3['sentiword_analysis'].apply(senti_label)
df3
#counting
blob_3=df3.groupby('senti_textblob').count()
print("textblob_step3:",blob_3["textblob_polarity"])

wordnet_3=df3.groupby('senti_wordnet').count()
print("wordnet_step3:",wordnet_3["sentiword_analysis"])

#using senti_label_2
df3['senti_vader'] = df3['vader_score'].apply(senti_label_2)
vader_3=df3.groupby('senti_vader').count()
print('vader_step3:',vader_3["vader_score"])

In [ ]:
#new dfs
new_dfs=[df1,df2,df3]
new_dfs=pd.concat(new_dfs)
new_dfs=new_dfs.reset_index(drop=True)
len(new_dfs)
# new_dfs.to_csv("unsupervised_datasets/lexicon_all.csv")

In [ ]:
dfs['senti_textblob'] = dfs['textblob_polarity'].apply(senti_label)
dfs['senti_wordnet'] = dfs['sentiword_analysis'].apply(senti_label)
dfs
#counting
blob_all=dfs.groupby('senti_textblob').count()
print("textblob_all:",blob_all["textblob_polarity"])

wordnet_all=dfs.groupby('senti_wordnet').count()
print("wordnet_all:",wordnet_all["sentiword_analysis"])

#using senti_label_2
dfs['senti_vader'] = dfs['vader_score'].apply(senti_label_2)
vader_all=dfs.groupby('senti_vader').count()
print('vader_all:',vader_all["vader_score"])

In [ ]:
vader_all=dfs.groupby('senti_vader').count()
print(vader_all["vader_score"])
wordnet_all=dfs.groupby('senti_wordnet').count()
print(wordnet_all["sentiword_analysis"])
blob_all=dfs.groupby('senti_textblob').count()
print(blob_all["textblob_polarity"])

In [ ]:
#extreme positive
slight_pos_wordnet = new_dfs[new_dfs['sentiword_analysis'].between(0.5,1)]
print("wordnet:",len(slight_pos_wordnet))
slight_pos_blob = new_dfs[new_dfs['textblob_polarity'].between(0.5,1)]
print("textbolb",len(slight_pos_blob))
slight_pos_vader = new_dfs[new_dfs['vader_score'].between(0.5,1)]
print("vader",len(slight_pos_vader))
#extreme negative 
slight_pos_wordnet = new_dfs[new_dfs['sentiword_analysis'].between(-1,-0.5)]
print("wordnet:",len(slight_pos_wordnet))
slight_pos_blob = new_dfs[new_dfs['textblob_polarity'].between(-1,-0.5)]
print("textblob",len(slight_pos_blob))
slight_pos_vader = new_dfs[new_dfs['vader_score'].between(-1,-0.5)]
print("vader",len(slight_pos_vader))

In [ ]:
#for each city: we use mean value to show their sentiment orientation of each city
#for england, each step with three methods,counting the number of negative, positive and neutral
#step1--Textblob
#London
London_1 = df1[df1['near'].str.contains('London|london')]
London_1 = London_1.reset_index(drop=True)
#rows (include 1)/ all rows
london_pos_1=len(London_1[London_1['senti_textblob']==1])/len(London_1)
london_neg_1=len(London_1[London_1['senti_textblob']==-1])/len(London_1)
print("london_pos_1",round(london_pos_1,3))
print("london_neg_1",round(london_neg_1,3))

#Birmingham
Birmingham_1 = df1[df1['near'].str.contains('Birmingham|birmingham')]
Birmingham_1 = Birmingham_1.reset_index(drop=True)
#rows include 1 or -1/ all rows
Birmingham_pos_1=len(Birmingham_1[Birmingham_1['senti_textblob']==1])/len(Birmingham_1)
Birmingham_neg_1=len(Birmingham_1[Birmingham_1['senti_textblob']==-1])/len(Birmingham_1)
print("Birmingham_pos_1",round(Birmingham_pos_1,3))
print("Birmingham_neg_1",round(Birmingham_neg_1,3))

#Bristol
Bristol_1 = df1[df1['near'].str.contains('Bristol|bristol')]
Bristol_1 = Bristol_1.reset_index(drop=True)
Bristol_pos_1=len(Bristol_1[Bristol_1['senti_textblob']==1])/len(Bristol_1)
Bristol_neg_1=len(Bristol_1[Bristol_1['senti_textblob']==-1])/len(Bristol_1)
print("Bristol_pos_1",round(Bristol_pos_1,3))
print("Bristol_neg_1",round(Bristol_neg_1,3))

# #Leeds
Leeds_1 = df1[df1['near'].str.contains('Leeds|leeds')]
Leeds_1 = Leeds_1.reset_index(drop=True)
Leeds_pos_1=len(Leeds_1[Leeds_1['senti_textblob']==1])/len(Leeds_1)
Leeds_neg_1=len(Leeds_1[Leeds_1['senti_textblob']==-1])/len(Leeds_1)
print("Leeds_pos_1",round(Leeds_pos_1,3))
print("Leeds_neg_1",round(Leeds_neg_1,3))

# #Liverpool
Liverpool_1 = df1[df1['near'].str.contains('Liverpool|liverpool')]
Liverpool_1 = Liverpool_1.reset_index(drop=True)
Liverpool_pos_1=len(Liverpool_1[Liverpool_1['senti_textblob']==1])/len(Liverpool_1)
Liverpool_neg_1=len(Liverpool_1[Liverpool_1['senti_textblob']==-1])/len(Liverpool_1)
print("Liverpool_pos_1",round(Liverpool_pos_1,3))
print("Liverpool_neg_1",round(Liverpool_neg_1,3))

#Manchester
Manchester_1 = df1[df1['near'].str.contains('Manchester|manchester')]
Manchester_1 = Manchester_1.reset_index(drop=True)
Manchester_pos_1=len(Manchester_1[Manchester_1['senti_textblob']==1])/len(Manchester_1)
Manchester_neg_1=len(Manchester_1[Manchester_1['senti_textblob']==-1])/len(Manchester_1)
print("Manchester_pos_1",round(Manchester_pos_1,3))
print("Manchester_neg_1",round(Manchester_neg_1,3))

#Newcastle
Newcastle_1 = df1[df1['near'].str.contains('Newcastle|newcastle')]
Newcastle_1 = Newcastle_1.reset_index(drop=True)
Newcastle_pos_1=len(Newcastle_1[Newcastle_1['senti_textblob']==1])/len(Newcastle_1)
Newcastle_neg_1=len(Newcastle_1[Newcastle_1['senti_textblob']==-1])/len(Newcastle_1)
print("Newcastle_pos_1",round(Newcastle_pos_1,3))
print("Newcastle_neg_1",round(Newcastle_neg_1,3))

#Nottingham
Nottingham_1 = df1[df1['near'].str.contains('Nottingham|nottingham')]
Nottingham_1 = Nottingham_1.reset_index(drop=True)
Nottingham_pos_1=len(Nottingham_1[Nottingham_1['senti_textblob']==1])/len(Nottingham_1)
Nottingham_neg_1=len(Nottingham_1[Nottingham_1['senti_textblob']==-1])/len(Nottingham_1)
print("Nottingham_pos_1",round(Nottingham_pos_1,3))
print("Nottingham_neg_1",round(Nottingham_neg_1,3))

#Sheffield
Sheffield_1 = df1[df1['near'].str.contains('Sheffield|sheffield')]
Sheffield_1 = Sheffield_1.reset_index(drop=True)
Sheffield_pos_1=len(Sheffield_1[Sheffield_1['senti_textblob']==1])/len(Sheffield_1)
Sheffield_neg_1=len(Sheffield_1[Sheffield_1['senti_textblob']==-1])/len(Sheffield_1)
print("Sheffield_pos_1",round(Sheffield_pos_1,3))
print("Sheffield_neg_1",round(Sheffield_neg_1,3))

#Southampton
Southampton_1 = df1[df1['near'].str.contains('Southampton|southampton')]
Southampton_1 = Southampton_1.reset_index(drop=True)
Southampton_pos_1=len(Southampton_1[Southampton_1['senti_textblob']==1])/len(Southampton_1)
Southampton_neg_1=len(Southampton_1[Southampton_1['senti_textblob']==-1])/len(Southampton_1)
print("Southampton_pos_1",round(Southampton_pos_1,3))
print("Southampton_neg_1",round(Southampton_neg_1,3))

In [ ]:
#step2--Textblob
#London
London_2 = df2[df2['near'].str.contains('London|london')]
London_2 = London_2.reset_index(drop=True)
#rows include 1/ all rows
london_pos_2=len(London_2[London_2['senti_textblob']==1])/len(London_2)
london_neg_2=len(London_2[London_2['senti_textblob']==-1])/len(London_2)
print("london_pos_2",round(london_pos_2,3))
print("london_neg_2",round(london_neg_2,3))

#Birmingham
Birmingham_2 = df2[df2['near'].str.contains('Birmingham|birmingham')]
Birmingham_2 = Birmingham_2.reset_index(drop=True)
#rows include 1 or -1/ all rows
Birmingham_pos_2=len(Birmingham_2[Birmingham_2['senti_textblob']==1])/len(Birmingham_2)
Birmingham_neg_2=len(Birmingham_2[Birmingham_2['senti_textblob']==-1])/len(Birmingham_2)
print("Birmingham_pos_2",round(Birmingham_pos_2,3))
print("Birmingham_neg_2",round(Birmingham_neg_2,3))

#Bristol
Bristol_2 = df2[df2['near'].str.contains('Bristol|bristol')]
Bristol_2 = Bristol_2.reset_index(drop=True)
Bristol_pos_2=len(Bristol_2[Bristol_2['senti_textblob']==1])/len(Bristol_2)
Bristol_neg_2=len(Bristol_2[Bristol_2['senti_textblob']==-1])/len(Bristol_2)
print("Bristol_pos_2",round(Bristol_pos_2,3))
print("Bristol_neg_2",round(Bristol_neg_2,3))

# #Leeds
Leeds_2 = df2[df2['near'].str.contains('Leeds|leeds')]
Leeds_2 = Leeds_2.reset_index(drop=True)
Leeds_pos_2=len(Leeds_2[Leeds_2['senti_textblob']==1])/len(Leeds_2)
Leeds_neg_2=len(Leeds_2[Leeds_2['senti_textblob']==-1])/len(Leeds_2)
print("Leeds_pos_2",round(Leeds_pos_2,3))
print("Leeds_neg_2",round(Leeds_neg_2,3))

# #Liverpool
Liverpool_2 = df2[df2['near'].str.contains('Liverpool|liverpool')]
Liverpool_2 = Liverpool_2.reset_index(drop=True)
Liverpool_pos_2=len(Liverpool_2[Liverpool_2['senti_textblob']==1])/len(Liverpool_2)
Liverpool_neg_2=len(Liverpool_2[Liverpool_2['senti_textblob']==-1])/len(Liverpool_2)
print("Liverpool_pos_2",round(Liverpool_pos_2,3))
print("Liverpool_neg_2",round(Liverpool_neg_2,3))

#Manchester
Manchester_2 = df2[df2['near'].str.contains('Manchester|manchester')]
Manchester_2 = Manchester_2.reset_index(drop=True)
Manchester_pos_2=len(Manchester_2[Manchester_2['senti_textblob']==1])/len(Manchester_2)
Manchester_neg_2=len(Manchester_2[Manchester_2['senti_textblob']==-1])/len(Manchester_2)
print("Manchester_pos_2",round(Manchester_pos_2,3))
print("Manchester_neg_2",round(Manchester_neg_2,3))

#Newcastle
Newcastle_2 = df2[df2['near'].str.contains('Newcastle|newcastle')]
Newcastle_2 = Newcastle_2.reset_index(drop=True)
Newcastle_pos_2=len(Newcastle_2[Newcastle_2['senti_textblob']==1])/len(Newcastle_2)
Newcastle_neg_2=len(Newcastle_2[Newcastle_2['senti_textblob']==-1])/len(Newcastle_2)
print("Newcastle_pos_2",round(Newcastle_pos_2,3))
print("Newcastle_neg_2",round(Newcastle_neg_2,3))

#Nottingham
Nottingham_2 = df2[df2['near'].str.contains('Nottingham|nottingham')]
Nottingham_2 = Nottingham_2.reset_index(drop=True)
Nottingham_pos_2=len(Nottingham_2[Nottingham_2['senti_textblob']==1])/len(Nottingham_2)
Nottingham_neg_2=len(Nottingham_2[Nottingham_2['senti_textblob']==-1])/len(Nottingham_2)
print("Nottingham_pos_2",round(Nottingham_pos_2,3))
print("Nottingham_neg_2",round(Nottingham_neg_2,3))

#Sheffield
Sheffield_2 = df2[df2['near'].str.contains('Sheffield|sheffield')]
Sheffield_2 = Sheffield_2.reset_index(drop=True)
Sheffield_pos_2=len(Sheffield_2[Sheffield_2['senti_textblob']==1])/len(Sheffield_2)
Sheffield_neg_2=len(Sheffield_2[Sheffield_2['senti_textblob']==-1])/len(Sheffield_2)
print("Sheffield_pos_2",round(Sheffield_pos_2,3))
print("Sheffield_neg_2",round(Sheffield_neg_2,3))

#Southampton
Southampton_2 = df2[df2['near'].str.contains('Southampton|southampton')]
Southampton_2 = Southampton_2.reset_index(drop=True)
Southampton_pos_2=len(Southampton_2[Southampton_2['senti_textblob']==1])/len(Southampton_2)
Southampton_neg_2=len(Southampton_2[Southampton_2['senti_textblob']==-1])/len(Southampton_2)
print("Southampton_pos_2",round(Southampton_pos_2,3))
print("Southampton_neg_2",round(Southampton_neg_2,3))

In [ ]:
df3.isna().sum()

In [ ]:
#step3--Textblob
#London
London_3 = df3[df3['near'].str.contains('London|london')]
London_3 = London_3.reset_index(drop=True)
#rows include 1/ all rows
london_pos_3=len(London_3[London_3['senti_textblob']==1])/len(London_3)
#len(London_1[London_1['senti_textblob']==0])/len(London_1)
london_neg_3=len(London_3[London_3['senti_textblob']==-1])/len(London_3)
print("london_pos_3",round(london_pos_3,3))
print("london_neg_3",round(london_neg_3,3))

#Birmingham
Birmingham_3 = df3[df3['near'].str.contains('Birmingham|birmingham')]
Birmingham_3 = Birmingham_3.reset_index(drop=True)
#rows include 1 or -1/ all rows
Birmingham_pos_3=len(Birmingham_3[Birmingham_3['senti_textblob']==1])/len(Birmingham_3)
Birmingham_neg_3=len(Birmingham_3[Birmingham_3['senti_textblob']==-1])/len(Birmingham_3)
print("Birmingham_pos_3",round(Birmingham_pos_3,3))
print("Birmingham_neg_3",round(Birmingham_neg_3,3))

#Bristol
Bristol_3 = df3[df3['near'].str.contains('Bristol|bristol')]
Bristol_3 = Bristol_3.reset_index(drop=True)
Bristol_pos_3=len(Bristol_3[Bristol_3['senti_textblob']==1])/len(Bristol_3)
Bristol_neg_3=len(Bristol_3[Bristol_3['senti_textblob']==-1])/len(Bristol_3)
print("Bristol_pos_3",round(Bristol_pos_3,3))
print("Bristol_neg_3",round(Bristol_neg_3,3))

# #Leeds
Leeds_3 = df3[df3['near'].str.contains('Leeds|leeds')]
Leeds_3 = Leeds_3.reset_index(drop=True)
Leeds_pos_3=len(Leeds_3[Leeds_3['senti_textblob']==1])/len(Leeds_3)
Leeds_neg_3=len(Leeds_3[Leeds_3['senti_textblob']==-1])/len(Leeds_3)
print("Leeds_pos_3",round(Leeds_pos_3,3))
print("Leeds_neg_3",round(Leeds_neg_3,3))

# #Liverpool
Liverpool_3 = df3[df3['near'].str.contains('Liverpool|liverpool')]
Liverpool_3 = Liverpool_3.reset_index(drop=True)
Liverpool_pos_3=len(Liverpool_3[Liverpool_3['senti_textblob']==1])/len(Liverpool_3)
Liverpool_neg_3=len(Liverpool_3[Liverpool_3['senti_textblob']==-1])/len(Liverpool_3)
print("Liverpool_pos_3",round(Liverpool_pos_3,3))
print("Liverpool_neg_3",round(Liverpool_neg_3,3))

#Manchester
Manchester_3 = df3[df3['near'].str.contains('Manchester|manchester')]
Manchester_3 = Manchester_3.reset_index(drop=True)
Manchester_pos_3=len(Manchester_3[Manchester_3['senti_textblob']==1])/len(Manchester_3)
Manchester_neg_3=len(Manchester_3[Manchester_3['senti_textblob']==-1])/len(Manchester_3)
print("Manchester_pos_3",round(Manchester_pos_3,2))
print("Manchester_neg_3",round(Manchester_neg_3,2))

#Newcastle
Newcastle_3 = df3[df3['near'].str.contains('Newcastle|newcastle')]
Newcastle_3 = Newcastle_3.reset_index(drop=True)
Newcastle_pos_3=len(Newcastle_3[Newcastle_3['senti_textblob']==1])/len(Newcastle_3)
Newcastle_neg_3=len(Newcastle_3[Newcastle_3['senti_textblob']==-1])/len(Newcastle_3)
print("Newcastle_pos_3",round(Newcastle_pos_3,3))
print("Newcastle_neg_3",round(Newcastle_neg_3,3))

#Nottingham
Nottingham_3 = df3[df3['near'].str.contains('Nottingham|nottingham')]
Nottingham_3 = Nottingham_3.reset_index(drop=True)
Nottingham_pos_3=len(Nottingham_3[Nottingham_3['senti_textblob']==1])/len(Nottingham_3)
Nottingham_neg_3=len(Nottingham_3[Nottingham_3['senti_textblob']==-1])/len(Nottingham_3)
print("Nottingham_pos_3",round(Nottingham_pos_3,3))
print("Nottingham_neg_3",round(Nottingham_neg_3,3))

#Sheffield
Sheffield_3 = df3[df3['near'].str.contains('Sheffield|sheffield')]
Sheffield_3 = Sheffield_3.reset_index(drop=True)
Sheffield_pos_3=len(Sheffield_3[Sheffield_3['senti_textblob']==1])/len(Sheffield_3)
Sheffield_neg_3=len(Sheffield_3[Sheffield_3['senti_textblob']==-1])/len(Sheffield_3)
print("Sheffield_pos_3",round(Sheffield_pos_3,3))
print("Sheffield_neg_3",round(Sheffield_neg_3,3))

#Southampton
Southampton_3 = df3[df3['near'].str.contains('Southampton|southampton')]
Southampton_3 = Southampton_3.reset_index(drop=True)
Southampton_pos_3=len(Southampton_3[Southampton_3['senti_textblob']==1])/len(Southampton_3)
Southampton_neg_3=len(Southampton_3[Southampton_3['senti_textblob']==-1])/len(Southampton_3)
print("Southampton_pos_3",round(Southampton_pos_3,3))
print("Southampton_neg_3",round(Southampton_neg_3,3))

In [ ]:
#step1--VADER
#London
London_1 = df1[df1['near'].str.contains('London|london')]
London_1 = London_1.reset_index(drop=True)
#rows include 1/ all rows
london_pos_1=len(London_1[London_1['senti_vader']==1])/len(London_1)
#london_neu=len(London_1[London_1['senti_textblob']==0])/len(London_1)
london_neg_1=len(London_1[London_1['senti_vader']==-1])/len(London_1)
print("london_pos_1",round(london_pos_1,3))
print("london_neg_1",round(london_neg_1,3))
# print("london_neg_1",round(london_neu,3))
#Birmingham
Birmingham_1 = df1[df1['near'].str.contains('Birmingham|birmingham')]
Birmingham_1 = Birmingham_1.reset_index(drop=True)
#rows include 1 or -1/ all rows
Birmingham_pos_1=len(Birmingham_1[Birmingham_1['senti_vader']==1])/len(Birmingham_1)
Birmingham_neg_1=len(Birmingham_1[Birmingham_1['senti_vader']==-1])/len(Birmingham_1)
print("Birmingham_pos_1",round(Birmingham_pos_1,3))
print("Birmingham_neg_1",round(Birmingham_neg_1,3))

#Bristol
Bristol_1 = df1[df1['near'].str.contains('Bristol|bristol')]
Bristol_1 = Bristol_1.reset_index(drop=True)
Bristol_pos_1=len(Bristol_1[Bristol_1['senti_vader']==1])/len(Bristol_1)
Bristol_neg_1=len(Bristol_1[Bristol_1['senti_vader']==-1])/len(Bristol_1)
print("Bristol_pos_1",round(Bristol_pos_1,3))
print("Bristol_neg_1",round(Bristol_neg_1,3))

# #Leeds
Leeds_1 = df1[df1['near'].str.contains('Leeds|leeds')]
Leeds_1 = Leeds_1.reset_index(drop=True)
Leeds_pos_1=len(Leeds_1[Leeds_1['senti_vader']==1])/len(Leeds_1)
Leeds_neg_1=len(Leeds_1[Leeds_1['senti_vader']==-1])/len(Leeds_1)
print("Leeds_pos_1",round(Leeds_pos_1,3))
print("Leeds_neg_1",round(Leeds_neg_1,3))

#Liverpool
Liverpool_1 = df1[df1['near'].str.contains('Liverpool|liverpool')]
Liverpool_1 = Liverpool_1.reset_index(drop=True)
Liverpool_pos_1=len(Liverpool_1[Liverpool_1['senti_vader']==1])/len(Liverpool_1)
Liverpool_neg_1=len(Liverpool_1[Liverpool_1['senti_vader']==-1])/len(Liverpool_1)
print("Liverpool_pos_1",round(Liverpool_pos_1,3))
print("Liverpool_neg_1",round(Liverpool_neg_1,3))

#Manchester
Manchester_1 = df1[df1['near'].str.contains('Manchester|manchester')]
Manchester_1 = Manchester_1.reset_index(drop=True)
Manchester_pos_1=len(Manchester_1[Manchester_1['senti_vader']==1])/len(Manchester_1)
Manchester_neg_1=len(Manchester_1[Manchester_1['senti_vader']==-1])/len(Manchester_1)
print("Manchester_pos_1",round(Manchester_pos_1,3))
print("Manchester_neg_1",round(Manchester_neg_1,3))

#Newcastle
Newcastle_1 = df1[df1['near'].str.contains('Newcastle|newcastle')]
Newcastle_1 = Newcastle_1.reset_index(drop=True)
Newcastle_pos_1=len(Newcastle_1[Newcastle_1['senti_vader']==1])/len(Newcastle_1)
Newcastle_neg_1=len(Newcastle_1[Newcastle_1['senti_vader']==-1])/len(Newcastle_1)
print("Newcastle_pos_1",round(Newcastle_pos_1,3))
print("Newcastle_neg_1",round(Newcastle_neg_1,3))

#Nottingham
Nottingham_1 = df1[df1['near'].str.contains('Nottingham|nottingham')]
Nottingham_1 = Nottingham_1.reset_index(drop=True)
Nottingham_pos_1=len(Nottingham_1[Nottingham_1['senti_vader']==1])/len(Nottingham_1)
Nottingham_neg_1=len(Nottingham_1[Nottingham_1['senti_vader']==-1])/len(Nottingham_1)
print("Nottingham_pos_1",round(Nottingham_pos_1,3))
print("Nottingham_neg_1",round(Nottingham_neg_1,3))

#Sheffield
Sheffield_1 = df1[df1['near'].str.contains('Sheffield|sheffield')]
Sheffield_1 = Sheffield_1.reset_index(drop=True)
Sheffield_pos_1=len(Sheffield_1[Sheffield_1['senti_vader']==1])/len(Sheffield_1)
Sheffield_neg_1=len(Sheffield_1[Sheffield_1['senti_vader']==-1])/len(Sheffield_1)
print("Sheffield_pos_1",round(Sheffield_pos_1,3))
print("Sheffield_neg_1",round(Sheffield_neg_1,3))

#Southampton
Southampton_1 = df1[df1['near'].str.contains('Southampton|southampton')]
Southampton_1 = Southampton_1.reset_index(drop=True)
Southampton_pos_1=len(Southampton_1[Southampton_1['senti_vader']==1])/len(Southampton_1)
Southampton_neg_1=len(Southampton_1[Southampton_1['senti_vader']==-1])/len(Southampton_1)
print("Southampton_pos_1",round(Southampton_pos_1,3))
print("Southampton_neg_1",round(Southampton_neg_1,3))

In [ ]:
#step2--VADER
#London
London_2 = df2[df2['near'].str.contains('London|london')]
London_2 = London_2.reset_index(drop=True)
#rows include 1/ all rows
london_pos_2=len(London_2[London_2['senti_vader']==1])/len(London_2)
#len(London_1[London_1['senti_textblob']==0])/len(London_1)
london_neg_2=len(London_2[London_2['senti_vader']==-1])/len(London_2)
print("london_pos_2",round(london_pos_2,3))
print("london_neg_2",round(london_neg_2,3))

#Birmingham
Birmingham_2 = df2[df2['near'].str.contains('Birmingham|birmingham')]
Birmingham_2 = Birmingham_2.reset_index(drop=True)
#rows include 1 or -1/ all rows
Birmingham_pos_2=len(Birmingham_2[Birmingham_2['senti_vader']==1])/len(Birmingham_2)
Birmingham_neg_2=len(Birmingham_2[Birmingham_2['senti_vader']==-1])/len(Birmingham_2)
print("Birmingham_pos_2",round(Birmingham_pos_2,3))
print("Birmingham_neg_2",round(Birmingham_neg_2,3))

#Bristol
Bristol_2 = df2[df2['near'].str.contains('Bristol|bristol')]
Bristol_2 = Bristol_2.reset_index(drop=True)
Bristol_pos_2=len(Bristol_2[Bristol_2['senti_vader']==1])/len(Bristol_2)
Bristol_neg_2=len(Bristol_2[Bristol_2['senti_vader']==-1])/len(Bristol_2)
print("Bristol_pos_2",round(Bristol_pos_2,3))
print("Bristol_neg_2",round(Bristol_neg_2,3))

# #Leeds
Leeds_2 = df2[df2['near'].str.contains('Leeds|leeds')]
Leeds_2 = Leeds_2.reset_index(drop=True)
Leeds_pos_2=len(Leeds_2[Leeds_2['senti_vader']==1])/len(Leeds_2)
Leeds_neg_2=len(Leeds_2[Leeds_2['senti_vader']==-1])/len(Leeds_2)
print("Leeds_pos_2",round(Leeds_pos_2,3))
print("Leeds_neg_2",round(Leeds_neg_2,3))

# #Liverpool
Liverpool_2 = df2[df2['near'].str.contains('Liverpool|liverpool')]
Liverpool_2 = Liverpool_2.reset_index(drop=True)
Liverpool_pos_2=len(Liverpool_2[Liverpool_2['senti_vader']==1])/len(Liverpool_2)
Liverpool_neg_2=len(Liverpool_2[Liverpool_2['senti_vader']==-1])/len(Liverpool_2)
print("Liverpool_pos_2",round(Liverpool_pos_2,3))
print("Liverpool_neg_2",round(Liverpool_neg_2,3))

#Manchester
Manchester_2 = df2[df2['near'].str.contains('Manchester|manchester')]
Manchester_2 = Manchester_2.reset_index(drop=True)
Manchester_pos_2=len(Manchester_2[Manchester_2['senti_vader']==1])/len(Manchester_2)
Manchester_neg_2=len(Manchester_2[Manchester_2['senti_vader']==-1])/len(Manchester_2)
print("Manchester_pos_2",round(Manchester_pos_2,3))
print("Manchester_neg_2",round(Manchester_neg_2,3))

#Newcastle
Newcastle_2 = df2[df2['near'].str.contains('Newcastle|newcastle')]
Newcastle_2 = Newcastle_2.reset_index(drop=True)
Newcastle_pos_2=len(Newcastle_2[Newcastle_2['senti_vader']==1])/len(Newcastle_2)
Newcastle_neg_2=len(Newcastle_2[Newcastle_2['senti_vader']==-1])/len(Newcastle_2)
print("Newcastle_pos_2",round(Newcastle_pos_2,3))
print("Newcastle_neg_2",round(Newcastle_neg_2,3))

#Nottingham
Nottingham_2 = df2[df2['near'].str.contains('Nottingham|nottingham')]
Nottingham_2 = Nottingham_2.reset_index(drop=True)
Nottingham_pos_2=len(Nottingham_2[Nottingham_2['senti_vader']==1])/len(Nottingham_2)
Nottingham_neg_2=len(Nottingham_2[Nottingham_2['senti_vader']==-1])/len(Nottingham_2)
print("Nottingham_pos_2",round(Nottingham_pos_2,3))
print("Nottingham_neg_2",round(Nottingham_neg_2,3))

#Sheffield
Sheffield_2 = df2[df2['near'].str.contains('Sheffield|sheffield')]
Sheffield_2 = Sheffield_2.reset_index(drop=True)
Sheffield_pos_2=len(Sheffield_2[Sheffield_2['senti_vader']==1])/len(Sheffield_2)
Sheffield_neg_2=len(Sheffield_2[Sheffield_2['senti_vader']==-1])/len(Sheffield_2)
print("Sheffield_pos_2",round(Sheffield_pos_2,3))
print("Sheffield_neg_2",round(Sheffield_neg_2,3))

#Southampton
Southampton_2 = df2[df2['near'].str.contains('Southampton|southampton')]
Southampton_2 = Southampton_2.reset_index(drop=True)
Southampton_pos_2=len(Southampton_2[Southampton_2['senti_vader']==1])/len(Southampton_2)
Southampton_neg_2=len(Southampton_2[Southampton_2['senti_vader']==-1])/len(Southampton_2)
print("Southampton_pos_2",round(Southampton_pos_2,3))
print("Southampton_neg_2",round(Southampton_neg_2,3))

In [ ]:
#step3--VADER
#London
London_3 = df3[df3['near'].str.contains('London|london')]
London_3 = London_3.reset_index(drop=True)
#rows include 1/ all rows
london_pos_3=len(London_3[London_3['senti_vader']==1])/len(London_3)
#len(London_1[London_1['senti_textblob']==0])/len(London_1)
london_neg_3=len(London_3[London_3['senti_vader']==-1])/len(London_3)
print("london_pos_3",round(london_pos_3,3))
print("london_neg_3",round(london_neg_3,3))

#Birmingham
Birmingham_3 = df3[df3['near'].str.contains('Birmingham|birmingham')]
Birmingham_3 = Birmingham_3.reset_index(drop=True)
#rows include 1 or -1/ all rows
Birmingham_pos_3=len(Birmingham_3[Birmingham_3['senti_vader']==1])/len(Birmingham_3)
Birmingham_neg_3=len(Birmingham_3[Birmingham_3['senti_vader']==-1])/len(Birmingham_3)
print("Birmingham_pos_3",round(Birmingham_pos_3,3))
print("Birmingham_neg_3",round(Birmingham_neg_3,3))

#Bristol
Bristol_3 = df3[df3['near'].str.contains('Bristol|bristol')]
Bristol_3 = Bristol_3.reset_index(drop=True)
Bristol_pos_3=len(Bristol_3[Bristol_3['senti_vader']==1])/len(Bristol_3)
Bristol_neg_3=len(Bristol_3[Bristol_3['senti_vader']==-1])/len(Bristol_3)
print("Bristol_pos_3",round(Bristol_pos_3,3))
print("Bristol_neg_3",round(Bristol_neg_3,3))

# #Leeds
Leeds_3 = df3[df3['near'].str.contains('Leeds|leeds')]
Leeds_3 = Leeds_3.reset_index(drop=True)
Leeds_pos_3=len(Leeds_3[Leeds_3['senti_vader']==1])/len(Leeds_3)
Leeds_neg_3=len(Leeds_3[Leeds_3['senti_vader']==-1])/len(Leeds_3)
print("Leeds_pos_3",round(Leeds_pos_3,3))
print("Leeds_neg_3",round(Leeds_neg_3,3))

# #Liverpool
Liverpool_3 = df3[df3['near'].str.contains('Liverpool|liverpool')]
Liverpool_3 = Liverpool_3.reset_index(drop=True)
Liverpool_pos_3=len(Liverpool_3[Liverpool_3['senti_vader']==1])/len(Liverpool_3)
Liverpool_neg_3=len(Liverpool_3[Liverpool_3['senti_vader']==-1])/len(Liverpool_3)
print("Liverpool_pos_3",round(Liverpool_pos_3,3))
print("Liverpool_neg_3",round(Liverpool_neg_3,3))

#Manchester
Manchester_3 = df3[df3['near'].str.contains('Manchester|manchester')]
Manchester_3 = Manchester_3.reset_index(drop=True)
Manchester_pos_3=len(Manchester_3[Manchester_3['senti_vader']==1])/len(Manchester_3)
Manchester_neg_3=len(Manchester_3[Manchester_3['senti_vader']==-1])/len(Manchester_3)
print("Manchester_pos_3",round(Manchester_pos_3,2))
print("Manchester_neg_3",round(Manchester_neg_3,2))

#Newcastle
Newcastle_3 = df3[df3['near'].str.contains('Newcastle|newcastle')]
Newcastle_3 = Newcastle_3.reset_index(drop=True)
Newcastle_pos_3=len(Newcastle_3[Newcastle_3['senti_vader']==1])/len(Newcastle_3)
Newcastle_neg_3=len(Newcastle_3[Newcastle_3['senti_vader']==-1])/len(Newcastle_3)
print("Newcastle_pos_3",round(Newcastle_pos_3,3))
print("Newcastle_neg_3",round(Newcastle_neg_3,3))

#Nottingham
Nottingham_3 = df3[df3['near'].str.contains('Nottingham|nottingham')]
Nottingham_3 = Nottingham_3.reset_index(drop=True)
Nottingham_pos_3=len(Nottingham_3[Nottingham_3['senti_vader']==1])/len(Nottingham_3)
Nottingham_neg_3=len(Nottingham_3[Nottingham_3['senti_vader']==-1])/len(Nottingham_3)
print("Nottingham_pos_3",round(Nottingham_pos_3,3))
print("Nottingham_neg_3",round(Nottingham_neg_3,3))

#Sheffield
Sheffield_3 = df3[df3['near'].str.contains('Sheffield|sheffield')]
Sheffield_3 = Sheffield_3.reset_index(drop=True)
Sheffield_pos_3=len(Sheffield_3[Sheffield_3['senti_vader']==1])/len(Sheffield_3)
Sheffield_neg_3=len(Sheffield_3[Sheffield_3['senti_vader']==-1])/len(Sheffield_3)
print("Sheffield_pos_3",round(Sheffield_pos_3,3))
print("Sheffield_neg_3",round(Sheffield_neg_3,3))

#Southampton
Southampton_3 = df3[df3['near'].str.contains('Southampton|southampton')]
Southampton_3 = Southampton_3.reset_index(drop=True)
Southampton_pos_3=len(Southampton_3[Southampton_3['senti_vader']==1])/len(Southampton_3)
Southampton_neg_3=len(Southampton_3[Southampton_3['senti_vader']==-1])/len(Southampton_3)
print("Southampton_pos_3",round(Southampton_pos_3,3))
print("Southampton_neg_3",round(Southampton_neg_3,3))

In [ ]:
#step1--WordNet
#London
# senti_wordnet
#London
London_1 = df1[df1['near'].str.contains('London|london')]
London_1 = London_1.reset_index(drop=True)
#rows include 1/ all rows
london_pos_1=len(London_1[London_1['senti_wordnet']==1])/len(London_1)
#london_neu=len(London_1[London_1['senti_textblob']==0])/len(London_1)
london_neg_1=len(London_1[London_1['senti_wordnet']==-1])/len(London_1)
print("london_pos_1",round(london_pos_1,3))
print("london_neg_1",round(london_neg_1,3))
# print("london_neg_1",round(london_neu,3))
#Birmingham
Birmingham_1 = df1[df1['near'].str.contains('Birmingham|birmingham')]
Birmingham_1 = Birmingham_1.reset_index(drop=True)
#rows include 1 or -1/ all rows
Birmingham_pos_1=len(Birmingham_1[Birmingham_1['senti_wordnet']==1])/len(Birmingham_1)
Birmingham_neg_1=len(Birmingham_1[Birmingham_1['senti_wordnet']==-1])/len(Birmingham_1)
print("Birmingham_pos_1",round(Birmingham_pos_1,3))
print("Birmingham_neg_1",round(Birmingham_neg_1,3))

#Bristol
Bristol_1 = df1[df1['near'].str.contains('Bristol|bristol')]
Bristol_1 = Bristol_1.reset_index(drop=True)
Bristol_pos_1=len(Bristol_1[Bristol_1['senti_wordnet']==1])/len(Bristol_1)
Bristol_neg_1=len(Bristol_1[Bristol_1['senti_wordnet']==-1])/len(Bristol_1)
print("Bristol_pos_1",round(Bristol_pos_1,3))
print("Bristol_neg_1",round(Bristol_neg_1,3))

# #Leeds
Leeds_1 = df1[df1['near'].str.contains('Leeds|leeds')]
Leeds_1 = Leeds_1.reset_index(drop=True)
Leeds_pos_1=len(Leeds_1[Leeds_1['senti_wordnet']==1])/len(Leeds_1)
Leeds_neg_1=len(Leeds_1[Leeds_1['senti_wordnet']==-1])/len(Leeds_1)
print("Leeds_pos_1",round(Leeds_pos_1,3))
print("Leeds_neg_1",round(Leeds_neg_1,3))

#Liverpool
Liverpool_1 = df1[df1['near'].str.contains('Liverpool|liverpool')]
Liverpool_1 = Liverpool_1.reset_index(drop=True)
Liverpool_pos_1=len(Liverpool_1[Liverpool_1['senti_wordnet']==1])/len(Liverpool_1)
Liverpool_neg_1=len(Liverpool_1[Liverpool_1['senti_wordnet']==-1])/len(Liverpool_1)
print("Liverpool_pos_1",round(Liverpool_pos_1,3))
print("Liverpool_neg_1",round(Liverpool_neg_1,3))

#Manchester
Manchester_1 = df1[df1['near'].str.contains('Manchester|manchester')]
Manchester_1 = Manchester_1.reset_index(drop=True)
Manchester_pos_1=len(Manchester_1[Manchester_1['senti_wordnet']==1])/len(Manchester_1)
Manchester_neg_1=len(Manchester_1[Manchester_1['senti_wordnet']==-1])/len(Manchester_1)
print("Manchester_pos_1",round(Manchester_pos_1,3))
print("Manchester_neg_1",round(Manchester_neg_1,3))

#Newcastle
Newcastle_1 = df1[df1['near'].str.contains('Newcastle|newcastle')]
Newcastle_1 = Newcastle_1.reset_index(drop=True)
Newcastle_pos_1=len(Newcastle_1[Newcastle_1['senti_wordnet']==1])/len(Newcastle_1)
Newcastle_neg_1=len(Newcastle_1[Newcastle_1['senti_wordnet']==-1])/len(Newcastle_1)
print("Newcastle_pos_1",round(Newcastle_pos_1,3))
print("Newcastle_neg_1",round(Newcastle_neg_1,3))

#Nottingham
Nottingham_1 = df1[df1['near'].str.contains('Nottingham|nottingham')]
Nottingham_1 = Nottingham_1.reset_index(drop=True)
Nottingham_pos_1=len(Nottingham_1[Nottingham_1['senti_wordnet']==1])/len(Nottingham_1)
Nottingham_neg_1=len(Nottingham_1[Nottingham_1['senti_wordnet']==-1])/len(Nottingham_1)
print("Nottingham_pos_1",round(Nottingham_pos_1,3))
print("Nottingham_neg_1",round(Nottingham_neg_1,3))

#Sheffield
Sheffield_1 = df1[df1['near'].str.contains('Sheffield|sheffield')]
Sheffield_1 = Sheffield_1.reset_index(drop=True)
Sheffield_pos_1=len(Sheffield_1[Sheffield_1['senti_wordnet']==1])/len(Sheffield_1)
Sheffield_neg_1=len(Sheffield_1[Sheffield_1['senti_wordnet']==-1])/len(Sheffield_1)
print("Sheffield_pos_1",round(Sheffield_pos_1,3))
print("Sheffield_neg_1",round(Sheffield_neg_1,3))

#Southampton
Southampton_1 = df1[df1['near'].str.contains('Southampton|southampton')]
Southampton_1 = Southampton_1.reset_index(drop=True)
Southampton_pos_1=len(Southampton_1[Southampton_1['senti_wordnet']==1])/len(Southampton_1)
Southampton_neg_1=len(Southampton_1[Southampton_1['senti_wordnet']==-1])/len(Southampton_1)
print("Southampton_pos_1",round(Southampton_pos_1,3))
print("Southampton_neg_1",round(Southampton_neg_1,3))

In [ ]:
#step2--WordNet
#London
#step2--VADER
#London
London_2 = df2[df2['near'].str.contains('London|london')]
London_2 = London_2.reset_index(drop=True)
#rows include 1/ all rows
london_pos_2=len(London_2[London_2['senti_wordnet']==1])/len(London_2)
#len(London_1[London_1['senti_textblob']==0])/len(London_1)
london_neg_2=len(London_2[London_2['senti_wordnet']==-1])/len(London_2)
print("london_pos_2",round(london_pos_2,3))
print("london_neg_2",round(london_neg_2,3))

#Birmingham
Birmingham_2 = df2[df2['near'].str.contains('Birmingham|birmingham')]
Birmingham_2 = Birmingham_2.reset_index(drop=True)
#rows include 1 or -1/ all rows
Birmingham_pos_2=len(Birmingham_2[Birmingham_2['senti_wordnet']==1])/len(Birmingham_2)
Birmingham_neg_2=len(Birmingham_2[Birmingham_2['senti_wordnet']==-1])/len(Birmingham_2)
print("Birmingham_pos_2",round(Birmingham_pos_2,3))
print("Birmingham_neg_2",round(Birmingham_neg_2,3))

#Bristol
Bristol_2 = df2[df2['near'].str.contains('Bristol|bristol')]
Bristol_2 = Bristol_2.reset_index(drop=True)
Bristol_pos_2=len(Bristol_2[Bristol_2['senti_wordnet']==1])/len(Bristol_2)
Bristol_neg_2=len(Bristol_2[Bristol_2['senti_wordnet']==-1])/len(Bristol_2)
print("Bristol_pos_2",round(Bristol_pos_2,3))
print("Bristol_neg_2",round(Bristol_neg_2,3))

# #Leeds
Leeds_2 = df2[df2['near'].str.contains('Leeds|leeds')]
Leeds_2 = Leeds_2.reset_index(drop=True)
Leeds_pos_2=len(Leeds_2[Leeds_2['senti_wordnet']==1])/len(Leeds_2)
Leeds_neg_2=len(Leeds_2[Leeds_2['senti_wordnet']==-1])/len(Leeds_2)
print("Leeds_pos_2",round(Leeds_pos_2,3))
print("Leeds_neg_2",round(Leeds_neg_2,3))

# #Liverpool
Liverpool_2 = df2[df2['near'].str.contains('Liverpool|liverpool')]
Liverpool_2 = Liverpool_2.reset_index(drop=True)
Liverpool_pos_2=len(Liverpool_2[Liverpool_2['senti_wordnet']==1])/len(Liverpool_2)
Liverpool_neg_2=len(Liverpool_2[Liverpool_2['senti_wordnet']==-1])/len(Liverpool_2)
print("Liverpool_pos_2",round(Liverpool_pos_2,3))
print("Liverpool_neg_2",round(Liverpool_neg_2,3))

#Manchester
Manchester_2 = df2[df2['near'].str.contains('Manchester|manchester')]
Manchester_2 = Manchester_2.reset_index(drop=True)
Manchester_pos_2=len(Manchester_2[Manchester_2['senti_wordnet']==1])/len(Manchester_2)
Manchester_neg_2=len(Manchester_2[Manchester_2['senti_wordnet']==-1])/len(Manchester_2)
print("Manchester_pos_2",round(Manchester_pos_2,3))
print("Manchester_neg_2",round(Manchester_neg_2,3))

#Newcastle
Newcastle_2 = df2[df2['near'].str.contains('Newcastle|newcastle')]
Newcastle_2 = Newcastle_2.reset_index(drop=True)
Newcastle_pos_2=len(Newcastle_2[Newcastle_2['senti_wordnet']==1])/len(Newcastle_2)
Newcastle_neg_2=len(Newcastle_2[Newcastle_2['senti_wordnet']==-1])/len(Newcastle_2)
print("Newcastle_pos_2",round(Newcastle_pos_2,3))
print("Newcastle_neg_2",round(Newcastle_neg_2,3))

#Nottingham
Nottingham_2 = df2[df2['near'].str.contains('Nottingham|nottingham')]
Nottingham_2 = Nottingham_2.reset_index(drop=True)
Nottingham_pos_2=len(Nottingham_2[Nottingham_2['senti_wordnet']==1])/len(Nottingham_2)
Nottingham_neg_2=len(Nottingham_2[Nottingham_2['senti_wordnet']==-1])/len(Nottingham_2)
print("Nottingham_pos_2",round(Nottingham_pos_2,3))
print("Nottingham_neg_2",round(Nottingham_neg_2,3))

#Sheffield
Sheffield_2 = df2[df2['near'].str.contains('Sheffield|sheffield')]
Sheffield_2 = Sheffield_2.reset_index(drop=True)
Sheffield_pos_2=len(Sheffield_2[Sheffield_2['senti_wordnet']==1])/len(Sheffield_2)
Sheffield_neg_2=len(Sheffield_2[Sheffield_2['senti_wordnet']==-1])/len(Sheffield_2)
print("Sheffield_pos_2",round(Sheffield_pos_2,3))
print("Sheffield_neg_2",round(Sheffield_neg_2,3))

#Southampton
Southampton_2 = df2[df2['near'].str.contains('Southampton|southampton')]
Southampton_2 = Southampton_2.reset_index(drop=True)
Southampton_pos_2=len(Southampton_2[Southampton_2['senti_wordnet']==1])/len(Southampton_2)
Southampton_neg_2=len(Southampton_2[Southampton_2['senti_wordnet']==-1])/len(Southampton_2)
print("Southampton_pos_2",round(Southampton_pos_2,3))
print("Southampton_neg_2",round(Southampton_neg_2,3))

In [ ]:
#step3--Wordnet
#London
#London
London_3 = df3[df3['near'].str.contains('London|london')]
London_3 = London_3.reset_index(drop=True)
#rows include 1/ all rows
london_pos_3=len(London_3[London_3['senti_wordnet']==1])/len(London_3)
#len(London_1[London_1['senti_textblob']==0])/len(London_1)
london_neg_3=len(London_3[London_3['senti_wordnet']==-1])/len(London_3)
print("london_pos_3",round(london_pos_3,3))
print("london_neg_3",round(london_neg_3,3))

#Birmingham
Birmingham_3 = df3[df3['near'].str.contains('Birmingham|birmingham')]
Birmingham_3 = Birmingham_3.reset_index(drop=True)
#rows include 1 or -1/ all rows
Birmingham_pos_3=len(Birmingham_3[Birmingham_3['senti_wordnet']==1])/len(Birmingham_3)
Birmingham_neg_3=len(Birmingham_3[Birmingham_3['senti_wordnet']==-1])/len(Birmingham_3)
print("Birmingham_pos_3",round(Birmingham_pos_3,3))
print("Birmingham_neg_3",round(Birmingham_neg_3,3))

#Bristol
Bristol_3 = df3[df3['near'].str.contains('Bristol|bristol')]
Bristol_3 = Bristol_3.reset_index(drop=True)
Bristol_pos_3=len(Bristol_3[Bristol_3['senti_wordnet']==1])/len(Bristol_3)
Bristol_neg_3=len(Bristol_3[Bristol_3['senti_wordnet']==-1])/len(Bristol_3)
print("Bristol_pos_3",round(Bristol_pos_3,3))
print("Bristol_neg_3",round(Bristol_neg_3,3))

# #Leeds
Leeds_3 = df3[df3['near'].str.contains('Leeds|leeds')]
Leeds_3 = Leeds_3.reset_index(drop=True)
Leeds_pos_3=len(Leeds_3[Leeds_3['senti_wordnet']==1])/len(Leeds_3)
Leeds_neg_3=len(Leeds_3[Leeds_3['senti_wordnet']==-1])/len(Leeds_3)
print("Leeds_pos_3",round(Leeds_pos_3,3))
print("Leeds_neg_3",round(Leeds_neg_3,3))

# #Liverpool
Liverpool_3 = df3[df3['near'].str.contains('Liverpool|liverpool')]
Liverpool_3 = Liverpool_3.reset_index(drop=True)
Liverpool_pos_3=len(Liverpool_3[Liverpool_3['senti_wordnet']==1])/len(Liverpool_3)
Liverpool_neg_3=len(Liverpool_3[Liverpool_3['senti_wordnet']==-1])/len(Liverpool_3)
print("Liverpool_pos_3",round(Liverpool_pos_3,3))
print("Liverpool_neg_3",round(Liverpool_neg_3,3))

#Manchester
Manchester_3 = df3[df3['near'].str.contains('Manchester|manchester')]
Manchester_3 = Manchester_3.reset_index(drop=True)
Manchester_pos_3=len(Manchester_3[Manchester_3['senti_wordnet']==1])/len(Manchester_3)
Manchester_neg_3=len(Manchester_3[Manchester_3['senti_wordnet']==-1])/len(Manchester_3)
print("Manchester_pos_3",round(Manchester_pos_3,2))
print("Manchester_neg_3",round(Manchester_neg_3,2))

#Newcastle
Newcastle_3 = df3[df3['near'].str.contains('Newcastle|newcastle')]
Newcastle_3 = Newcastle_3.reset_index(drop=True)
Newcastle_pos_3=len(Newcastle_3[Newcastle_3['senti_wordnet']==1])/len(Newcastle_3)
Newcastle_neg_3=len(Newcastle_3[Newcastle_3['senti_wordnet']==-1])/len(Newcastle_3)
print("Newcastle_pos_3",round(Newcastle_pos_3,3))
print("Newcastle_neg_3",round(Newcastle_neg_3,3))

#Nottingham
Nottingham_3 = df3[df3['near'].str.contains('Nottingham|nottingham')]
Nottingham_3 = Nottingham_3.reset_index(drop=True)
Nottingham_pos_3=len(Nottingham_3[Nottingham_3['senti_wordnet']==1])/len(Nottingham_3)
Nottingham_neg_3=len(Nottingham_3[Nottingham_3['senti_wordnet']==-1])/len(Nottingham_3)
print("Nottingham_pos_3",round(Nottingham_pos_3,3))
print("Nottingham_neg_3",round(Nottingham_neg_3,3))

#Sheffield
Sheffield_3 = df3[df3['near'].str.contains('Sheffield|sheffield')]
Sheffield_3 = Sheffield_3.reset_index(drop=True)
Sheffield_pos_3=len(Sheffield_3[Sheffield_3['senti_wordnet']==1])/len(Sheffield_3)
Sheffield_neg_3=len(Sheffield_3[Sheffield_3['senti_wordnet']==-1])/len(Sheffield_3)
print("Sheffield_pos_3",round(Sheffield_pos_3,3))
print("Sheffield_neg_3",round(Sheffield_neg_3,3))

#Southampton
Southampton_3 = df3[df3['near'].str.contains('Southampton|southampton')]
Southampton_3 = Southampton_3.reset_index(drop=True)
Southampton_pos_3=len(Southampton_3[Southampton_3['senti_wordnet']==1])/len(Southampton_3)
Southampton_neg_3=len(Southampton_3[Southampton_3['senti_wordnet']==-1])/len(Southampton_3)
print("Southampton_pos_3",round(Southampton_pos_3,3))
print("Southampton_neg_3",round(Southampton_neg_3,3))

# Supervised Approaches

In [ ]:
import pandas as pd
import numpy as np
import gensim 
from gensim.models import Word2Vec
from nltk.tokenize import sent_tokenize, word_tokenize 
import warnings
warnings.filterwarnings("ignore")

In [ ]:
#df = pd.read_csv('supervised_sample_datasets/lexicon_step1.csv')
df = supervised_lexicon_step1
df

In [ ]:
#sample data
positive_df = df[df["senti_vader"] == 1]
positive_df = positive_df[:1000] #1000 positive sentiment
neutral_df = df[df["senti_vader"] == 0]
neutral_df = neutral_df[:1000] #1000 neutral sentiment
negative_df = df[df["senti_vader"] == -1]
negative_df = negative_df[:1000] #1000 neutral sentiment

In [ ]:
df=[positive_df, neutral_df, negative_df]
df=pd.concat(df)
df=df.reset_index(drop=True)
df.to_csv('sample_data.csv', index=False, encoding='utf_8_sig')

# Modelling

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import pandas as pd
import numpy as np
import gensim 
from gensim.models import Word2Vec
from nltk.tokenize import sent_tokenize, word_tokenize 
import warnings
warnings.filterwarnings("ignore")
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/sample_data.csv')

In [ ]:
#after manual check
negative_num=len(df[df['senti_vader'] < 0])
print("negative:", negative_num)
neutral_num=len(df[df['senti_vader'] == 0])
print("neutral", neutral_num)
positive_num=len(df[df['senti_vader'] > 0])
print("positive", positive_num) 

In [ ]:
#BoW
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer 
from nltk.tokenize import word_tokenize

def bag_of_words(df):
#     bow_vectorizer = CountVectorizer(max_df=0.90, min_df=0.2, stop_words=None, tokenizer=word_tokenize) 
    bow_vectorizer = CountVectorizer() 
    bow = bow_vectorizer.fit_transform(df['lemma_sentence(with POS)']) 
    #print(bow_vectorizer.get_feature_names())
    #print(bow_vectorizer.vocabulary_)
    return bow

df_bow=bag_of_words(df)
df_bow.shape

In [ ]:
#TF-IDF
def tf_idf(df):
#     tf_idf_vectorizer = TfidfVectorizer(max_df=0.9, min_df=0.2, stop_words=None, tokenizer=word_tokenize, norm='l2') 
    tf_idf_vectorizer = TfidfVectorizer(norm='l2') #extract features
    tfidf = tf_idf_vectorizer.fit_transform(df['lemma_sentence(with POS)']) #vectors
    return tfidf
df_tfidf=tf_idf(df)
df_tfidf.shape

In [ ]:
#Word2vec
#reference：https://www.pythonf.cn/read/93491
#https://github.com/Shwetago/Sentiment_Analysis/blob/master/Twitter_Sentiment_Analysis.ipynb
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize
Tokenize_tweet = df['lemma_sentence(with POS)'].apply(word_tokenize)
print(Tokenize_tweet)

Model_W2V = gensim.models.Word2Vec(Tokenize_tweet, vector_size=200, #features
                                   window=5, 
                                   min_count=1, 
                                   sg=1,  #skip-gram model
                                   hs=0,
                                   negative=10, 
                                   workers=2, 
                                   seed=34) 

In [ ]:
#Each word can get its own vector. The representation of a tweets can the vector sum of each word divided by the total number(average) 
#or just the sum of each word vector
def word2vec_tweet(tokens, size):
    vector=np.zeros(size).reshape((1,size))
    vector_cnt = 0
    for word in tokens:
        vector += Model_W2V.wv[word].reshape((1, size))
        vector_cnt += 1
    return vector/vector_cnt  #average for tweets

def word2vec_tweet_2(tokens, size):
    vector=np.zeros(size).reshape((1,size))
    vector_cnt = 0
    for word in tokens:
        vector += Model_W2V.wv[word].reshape((1, size))
    return vector  #sum of tweets

tweet_arr=np.zeros((len(Tokenize_tweet), 200))

for i in range (len(Tokenize_tweet)):
    tweet_arr[i,:] = word2vec_tweet(Tokenize_tweet[i], 200)
tweet_vec_df = pd.DataFrame(tweet_arr)
tweet_vec_df